In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import cm
import numpy as np

In [ ]:
%matplotlib notebook

### The 2 csv files

2 data files were used to simulate all the parameters of the experiment. The first contains made up data that will be visualized live and regard the first cycles of the experiment. A complete image of the experiment is given in the second file, which is based on real data from BEXUS 24 and data given by our professor Dr Mpalis.

In [ ]:
live_df = pd.read_excel('../Real-time analysis/XLSXs/First_Cycles.xlsx',
                        names=['time', 'P_in', 'P_out', 'T_in', 'T_out', 'Hum_in',
                               'Hum_out', 'CO2_V1', 'CO2_V2', 'O3_WE', 'O3_AE',
                               'Altitude','flags'],
                      header=0,
                      usecols=[i for i in range(0,13)])

In [ ]:
live_df

In [ ]:
button_df = pd.read_excel('../Real-time analysis/XLSXs/Sample CSV1.xlsx',
                       names=['time', 'P_in', 'P_out', 'T_in', 'T_out', 'Hum_in',
                              'Hum_out', 'CO2_V1', 'CO2_V2', 'O3_WE', 'O3_AE',
                              'Altitude','flags','flowrate','CO2','O3','CoM','SB_temp'],
                       header=0,
                       usecols=[i for i in range(0,18)])

In [ ]:
button_df.head(15)

### Functions for data handling

In [ ]:
def O3Concentration(df):
    O3_WE = df.O3_WE
    O3_AE = df.O3_AE
    T_in = df.T_in 
    
    a = 1.64
    WEe = 37
    AEe = 25
    b = 1 
    WEc = O3_WE - WEe - a*(O3_AE - AEe)
    
    """Fitting & a=a(T_in)"""
    return b * WEc    

In [ ]:
def CO2Concentration(df):
    CO2_V1 = df.CO2_V1
    CO2_V2 = df.CO2_V2
    T_in = df.T_in
    
    a= 1.52
    n= 0.724 # linearization coefficients calculated in tests (using fittings)
    Z = 1.33 
    S = 0.3604   
    Tcal = 19.78
    apos = 0.000556
    aneg = 0.000438
    bpos = 0.124
    bneg = 0.219
    
    """"TOWATCH Fitting Temp"""
    
    NR = CO2_V1/(Z*CO2_V2)
    
    if (T_in>Tcal):
        NRcomp = NR*(1+apos*(T_in-Tcal))
        Scomp = S + bpos*(T_in-Tcal)/Tcal
    else:
        NRcomp = NR*(1+aneg*(T_in-Tcal))
        Scomp = S + bneg*(T_in-Tcal)/Tcal
    
    if (1 - NRcomp > 0):
        result = ((-1/a)*np.log(1-(1-NRcomp)/Scomp)) **(1/n)
    else:
        result = -((-1/a)*np.log(1-(1-NRcomp)*(-1/Scomp))) **(1/n)
    return result

In [ ]:
V_in = 2

def flowrate(df):
    flow_rate=[]
    j=0
    
    for k in df['time']:
        try:
            dt = df.loc[j+1, 'time'] - k
            
            dpin=df.loc[j+1, 'P_in'] - df.loc[j, 'P_in']
            
            #dtin=df.loc[j+1, 'T_in'] - df.loc[j, 'T_in']
            
            dtin=0
            pout=(df.loc[j+1, 'P_out'] + df.loc[j, 'P_out'])/2
            pin=(df.loc[j+1, 'P_in'] + df.loc[j, 'P_in'])/2
            
            tout=273+(df.loc[j+1, 'T_out'] + df.loc[j, 'T_out'])/2
            tin=273+(df.loc[j+1, 'T_in'] + df.loc[j, 'T_in'])/2
            
            flow=(V_in/pout)*(tout/tin)*(dpin/dt-(pin/tin)*dtin/dt)*60
            
            flow_rate.append(flow if flow>0 else None)
            
            j+=1
        except:
            continue
        
    return pd.Series(flow_rate, dtype='float64')

### Adding extra columns in dataframes

In [ ]:
live_df['Flowrate'] = flowrate(live_df.mask(lambda x: x['flags']!=1))

live_df

## Plot handlers

In [ ]:
def flow_rate_plot(df):
    plt.style.use('seaborn')
    
    flow_rate_figure, ax1 = plt.subplots()
    
    
    ax1.scatter(df['time'], df['Flowrate'], s=20, 
                marker= '.', label = "ECO-WISE 2021")
    
    ax1.set_title("Pump's flowrate")
    ax1.set_xlabel("Time (sec)")
    ax1.set_ylabel("Flowrate (L/min)")
    ax1.legend()
    plt.tight_layout()
    
    return flow_rate_figure

In [ ]:
def temp_press_out_plot(df):
    
    temp_press_out_figure, ax3 = plt.subplots()
    
    plt.style.use('seaborn')
    
    colors=df['T_out']
    
    plot = ax3.scatter(df['P_out'], df['Altitude'], s=20, c=colors, cmap='jet', 
                  label = "ECO-WISE 2021",marker= '.')
    temp_press_out_figure.colorbar(plot, ax=ax3,label = 'Temperature out (°C)')
    
    
    ax3.set_title("Environmental variables")
    ax3.set_xlabel("Pressure (mbar)")
    ax3.set_ylabel("Altitude (m)")
    ax3.legend()
    plt.tight_layout()
    
    #ERRORS
    
    p = 1/100 #Error margin of  Pressure
    
    y_errormin = p*df['P_out']
    y_errormax = p*df['P_out']
    y_error = [y_errormin,y_errormax]
    
    x_error= 0
    
    ax3.errorbar(df['P_out'], 
                 df['Altitude'], 
                 yerr = y_error,
                 xerr = x_error, 
                 fmt=' ',
                 elinewidth=1,
                 capsize=5,
                 errorevery=100, 
                 capthick=1)  #ERROR EVERY to be shown in graph


    return temp_press_out_figure

In [ ]:
def humidity_plot(df):
    humidity_figure, (hum_in,hum_out) = plt.subplots(nrows=2, ncols=1, sharex=True)
    
    #hum_in.grid(linewidth= 0.5, linestyle= '--', color= '#262626', alpha= 0.2)
    #hum_out.grid(linewidth= 0.5, linestyle= '--', color= '#262626', alpha= 0.2)
    #plt.rcParams['axes.facecolor'] = '#ccccff'
    plt.style.use('seaborn')
    
    hum_in.plot(df['time'], df['Hum_in'], label = "Hum in (ECO-WISE 2021)")
    hum_out.plot(df['time'], df['Hum_out'], label = "Hum out (ECO-WISE 2021)")
    
    hum_in.set_title("Humidity")
    hum_in.set_ylabel("Humidity Inside (%)")  
    hum_in.legend()
    
    hum_out.set_xlabel("Time (min)")
    hum_out.set_ylabel("Humidity Outside (%)")   
    hum_out.legend()
    
    plt.tight_layout()
    
    return humidity_figure

In [ ]:
def O3_conc(df):
    
    O3_conc_figure, ax5 = plt.subplots()
    
    plt.style.use('seaborn')
    ax5.scatter(df['O3'], df['Altitude'], s=20,marker= '.', 
                  label = "ECO-WISE 2021")
    
    ax5.set_title("O3 concentration")
    ax5.set_xlabel("O3 (ppb)")
    ax5.set_ylabel("Altitude (m)")
    ax5.legend()
    plt.tight_layout()

    return O3_conc_figure

In [ ]:
def CO2_conc(df):
    
    CO2_conc_figure, ax6 = plt.subplots()
    
    plt.style.use('seaborn')
    ax6.scatter(df['CO2'], df['Altitude'], s=20, 
                marker= '.', label = "ECO-WISE 2021")
    
    ax6.set_title("CO2 concentration")
    ax6.set_xlabel("CO2 (v/v %)")
    ax6.set_ylabel("Altitude (m)")
    ax6.legend()
    plt.tight_layout()

    return CO2_conc_figure

In [ ]:
def altitude_time(df):
    
    altitude_time_figure, ax7 = plt.subplots()
    
    colors=df['T_out']
    
    plt.style.use('seaborn')
    plot = ax7.scatter(df['time'], df['Altitude'], s=20, c=colors, cmap='jet', 
                marker= '.', label = "ECO-WISE 2021")
    
    altitude_time_figure.colorbar(plot, ax=ax7,label = 'Temperature out (°C)')
    
    ax7.set_title("Balloon altitude (Altitude Over Time)")
    ax7.set_xlabel("Time (min)")
    ax7.set_ylabel("Altitude (m)")
    ax7.legend()
    plt.tight_layout()
    
    return altitude_time_figure

In [ ]:
def temp_press_in_plot(df):
    
    temp_press_in_figure, ax8 = plt.subplots()
    
    colors=df['T_in']
    
    plt.style.use('seaborn')
    plot = ax8.scatter(df['time'],df['P_in'], s=20, c=colors, cmap='jet', 
                marker= '.', label = "ECO-WISE 2021")
    
    temp_press_in_figure.colorbar(plot, ax=ax8,label = 'Temperature in (°C)')
    
    ax8.set_title("Sensor box variables")
    ax8.set_xlabel("Time (sec)")
    ax8.set_ylabel("Pressure (mbar)")
    ax8.legend()
    plt.tight_layout()

    return temp_press_in_figure

## Start Plotting

### Live Plots

In [ ]:
# %matplotlib notebook
plt.style.use('seaborn')
from matplotlib.animation import FuncAnimation
from mpl_toolkits.axes_grid1 import make_axes_locatable

def sensorbox(i):
    df = pd.read_csv('./data.csv', usecols=[3,0,1])
    
    colors = df['T_in']
    
    ax8.cla()
    plot = ax8.scatter(df['time'], df['P_in'], s=30, c=colors, cmap='jet', 
                marker= '.', label = "ECO-WISE 2021")
    ax8.set_title("Sensor box variables")
    ax8.set_xlabel("Time (sec)")
    ax8.set_ylabel("Pressure (mbar)")
    ax8.legend()
    
    cax.cla()
    fig.colorbar(plot, ax=ax8,cax=cax)
    
fig = plt.figure()
ax8 = fig.add_subplot(111)

div = make_axes_locatable(ax8)
cax = div.append_axes('right', '5%', '5%')

ani = FuncAnimation(fig, sensorbox, interval=1000)


A live graph of the sensor box variables (Pressure and Temperature over Time) is a useful way of confirming that the sensors are operating smoothly. There is a predicted pattern of pressure over time knowing the sensor box’s different stages. (It should be mentioned that these presented measurements are the mean of two different sets of sensors placed in different locations inside the box to increase the validity and accuracy of the measurements)

In [ ]:
def humidities(i):
    df = pd.read_csv('./data.csv', usecols=[0,5,6])
    
    hum_in.cla()
    hum_out.cla()
    
    hum_in.plot(df['time'], df['Hum_in'], label = "Hum in (ECO-WISE 2021)")
    hum_out.plot(df['time'], df['Hum_out'], label = "Hum out (ECO-WISE 2021)")
    
    hum_in.set_title("Humidity")
    hum_in.set_ylabel("Humidity Inside (%)")  
    hum_in.legend()
    
    hum_out.set_xlabel("Time (min)")
    hum_out.set_ylabel("Humidity Outside (%)")   
    hum_out.legend()

humidity_figure, (hum_in,hum_out) = plt.subplots(nrows=2, ncols=1, sharex=True)

ani = FuncAnimation(humidity_figure, humidities, interval=1000)

In [ ]:
def flowplot(i):
    df = pd.read_csv('./data.csv', usecols=[0,1,2,3,4,12])
    df['Flowrate'] = flowrate(df.mask(lambda x: x['flags']!=1))

    # print(df)
    ax1.cla()
    
    ax1.scatter(df['time'], df['Flowrate'], s=20, 
                marker= '.', label = "ECO-WISE 2021")
    
    ax1.set_title("Pump's flowrate")
    ax1.set_xlabel("Time (sec)")
    ax1.set_ylabel("Flowrate (L/min)")
    ax1.legend()

flow_rate_figure, ax1 = plt.subplots()

ani = FuncAnimation(flow_rate_figure, flowplot, interval=1000)

The pump flowrate is calculated using a theoretical model (Outside and inside temperature and pressure need to be measured) the validity of which has not been verified yet. During the process of calibrating the sensors a flowrate-meter will be used, and its measurements will be compared to the predicted values given by this model.

### Plots renewed using buttons

In [ ]:
temp_out_plot = temp_press_out_plot(button_df.loc[:,['T_out','P_out','Altitude']])

Outside temperature and pressure over altitude are measured in order to calculate the flowrate of the air pump. Additionally, it was determined that graphing this data using a color bar to indicate temperature is an aesthetically pleasing and compact way to present some important information.

In [ ]:
altitude_plot= altitude_time(button_df.loc[:,['time','Altitude','T_out']])

The Balloon’s Altitude is measured using both a GPS and an Altimeter. Due to the experiment’s requirements, the altimeter’s measurements are used (Increased accuracy. In fact, as with all other sensors, two altimeters are used). Again plotting the temperature on the graph is a compact way to add a third variable in a 2D graph.

In [ ]:
O3_plot = O3_conc(button_df.loc[:,['O3','Altitude']])

In [ ]:
CO2_plot = CO2_conc(button_df.loc[:,['CO2','Altitude']])

The concentration of CO2 and Ο3 in the atmosphere (over altitude) is the primary target of this experiment so it is imperative that these calculated values are presented in a real-time graph 